# Keras

TBD..

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=5)

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:,.5f}'.format

print("pandas ver={}".format(pd.__version__))
print("numpy ver={}".format(np.__version__))

pandas ver=0.24.1
numpy ver=1.16.2


In [27]:
from keras import layers, models

# Function API를 사용
def dnn(n_i, n_hs, n_o):
    i = layers.Input(shape = (n_i,))
    for l, n_h in enumerate(n_hs):
        if l == 0:
            h = layers.Dense(n_h, activation = 'relu')(i)            
        else:
            h = layers.Dense(n_h, activation = 'relu')(h)    
    o = layers.Dense(n_o, activation = 'softmax')(h)
    m = models.Model(i, o)
    m.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return m

def dnn(n_i, n_hs, n_o):
    m = models.Sequential()
    m.add(layers.Dense(n_i, activation='relu', input_shape=(n_i,)))
    for l, n_h in enumerate(n_hs):
        if l == 0:
            m.add(layers.Dense(n_h, activation='relu', input_shape=(n_i,)))
        else:            
            m.add(layers.Dense(n_h, activation='relu'))
    m.add(layers.Dense(n_o, activation = 'softmax'))
    m.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return m


In [11]:
from keras.datasets import mnist

# MNIST 샘플 데이터
(np_train_xs, np_train_ys), (np_test_xs, np_test_ys) = mnist.load_data()

In [12]:
r, w, h = np_train_xs.shape
print("r={}, w={}, h={}".format(r, w, h))

# 전처리 - 28x28 2D 이미지를 DNN 모델링을 위해 784 1D 로 변환합니다. 
dim_x = w * h
np_train_xs = np_train_xs.reshape(-1, dim_x).astype('float32')
np_test_xs = np_test_xs.reshape(-1, dim_x).astype('float32')
print(np_train_xs.shape)
print(np_test_xs.shape)

r=60000, w=28, h=28
(60000, 784)
(10000, 784)


In [13]:
# 전처리 - 0~1 사이로 데이터 Min Max Scaling
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
np_train_xs = scaler.fit_transform(np_train_xs)
np_test_xs = scaler.transform(np_test_xs)

In [14]:
# 전처리 - 레이블을 Ont Hot 인코팅 
from keras.utils import np_utils

n_class = 10
np_train_ys = np_utils.to_categorical(np_train_ys, n_class)
np_test_ys = np_utils.to_categorical(np_test_ys, n_class)
print(np_train_ys.shape)
print(np_train_ys[:3])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [26]:
model = dnn(n_i = dim_x, n_hs = [384, 128, 64], n_o = n_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 384)               301440    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               49280     
_________________________________________________________________
dense_21 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_22 (Dense)             (None, 10)                650       
Total params: 359,626
Trainable params: 359,626
Non-trainable params: 0
_________________________________________________________________


In [28]:
model = dnn(n_i = dim_x, n_hs = [384, 128, 64], n_o = n_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_24 (Dense)             (None, 384)               301440    
_________________________________________________________________
dense_25 (Dense)             (None, 128)               49280     
_________________________________________________________________
dense_26 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_27 (Dense)             (None, 10)                650       
Total params: 975,066
Trainable params: 975,066
Non-trainable params: 0
_________________________________________________________________


In [29]:
history = model.fit(np_train_xs, np_train_ys, epochs=10, batch_size=100, validation_split=0.2)
eval = model.evaluate(np_test_xs, np_test_ys, batch_size=100)
print(eval)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 4s 78us/step - loss: 0.2714 - acc: 0.9153 - val_loss: 0.1277 - val_acc: 0.9613
Epoch 2/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0984 - acc: 0.9699 - val_loss: 0.1098 - val_acc: 0.9699
Epoch 3/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0697 - acc: 0.9798 - val_loss: 0.1064 - val_acc: 0.9724
Epoch 4/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0511 - acc: 0.9849 - val_loss: 0.1040 - val_acc: 0.9757
Epoch 5/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0431 - acc: 0.9878 - val_loss: 0.1122 - val_acc: 0.9767
Epoch 6/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0339 - acc: 0.9903 - val_loss: 0.1240 - val_acc: 0.9745
Epoch 7/10
48000/48000 [==============================] - 3s 72us/step - loss: 0.0289 - acc: 0.9916 - val_loss: 0.1205 - val_acc